This notebook contains functions for investigating the natural power monoid, PN = P_{fin,0}(\NN).
Created by: Austin Antoniou

References:

[1] https://docs.python.org/3/library/itertools.html

[2] http://jeromekelleher.net/generating-integer-partitions.html

In [15]:
#packages needed
import itertools as it
from time import time
from numba import njit
import math as math

In [2]:
#Functions for subsets

def powerset(X): #returns generator of all subsets of X.  Code borrowed from [1]
    s = list(X)
    return it.chain.from_iterable(map(set,it.combinations(s, r)) for r in range(len(s)+1))

def npowerset(X): #returns generator of all nonempty subsets of X.
    s = list(X)
    return it.chain.from_iterable(map(set,it.combinations(s, r)) for r in range(1,len(s)+1))

def subsets_cont(X,Y): #returns subsets of X containing Y
    X0 = X - Y
    for S in powerset(X0):
        yield S.union(Y)

def nonunits(X): #returns subsets of X containing 0 and some other element
    for S in npowerset(X-{0}):
        yield S.union({0})

def interval_subsets(n): #returns generator of subsets of [0,n] containing the endpoints
    for S in powerset(set(range(1,n))):
        yield S.union({0,n})

def is_subset(X,Y): #decides whether X is a subset of Y
    for x in X:
        if not x in Y:
            return False
    return True

In [3]:
#Functions for setwise sums

def setsum(X,Y): #returns X+Y={x+y: (x,y) in (X,Y)}
    return set(x+y for x in X for y in Y)

def msum(*listofsets): #n-ary version of setsum
    S = listofsets[0]
    if len(listofsets) == 1:
        return S
    else:
        for i in range(1,len(listofsets)):
            S = setsum(S,listofsets[i])
        return S

def translate(X,c): #translates X by c
    return set(x+c for x in X)

def dilate(X,k): #dilates X by k
    return set(k*x for x in X)

def subsums(X): #returns set of subsums of a multiset or tuple X
    S = {0}
    for x in X:
        S = setsum(S,{0,x})
    return S

In [4]:
#Functions for testing sumset decompositions involving a given factor

def cofactor(X,A): #returns the largest possible B with X = A + B
    B = X
    for a in A:
        B = B & translate(X,-a)
    return B

#TODO def vitals(X,A): returns set of b contained in any B such that X = A+B

#TODO def cofactors(X,A): returns generator of all B with X = A+B

def divides(X,A): #decides whether A divides X
    return (X == setsum(A,cofactor(X,A)))

In [5]:
#Functions for testing irreducibility and finding atoms

atom_ref_path = 'atoms/atoms_max'

def is_atom(X): #decides whether X is an atom by testing divisibility by all its subsets
    for Y in nonunits(X-{max(X)}):
        if divides(X,Y):
            return False
    return True

def atoms_in(X): #returns generator of atoms which are subsets of X
    for Y in nonunits(X):
        if is_atom(Y):
            yield Y
            
def atoms_dividing(X):
    for A in atoms_in(X):
        if divides(X,A):
            yield A

def record_atoms(n): #records a list of the atoms with max n
    file = open(atom_ref_path+str(n),'w')
    for S in interval_subsets(n):
        if is_atom(S):
            file.write(str(S)+'\n')
    file.close()

def atom_ref(n): #opens the file containing line-by-line strings of the atoms with max n
    return open(atom_ref_path+str(n),'r')

def atom_gen(n):
    for a in atom_ref(n):
        yield tuple(sorted(eval(a)))

def atoms_in_gen(n,X):
    for a in atom_gen(n):
        if is_subset(a,X):
            yield a

In [6]:
###Now that we've recorded atoms up to a given max, we can use this to check irreducibility of subsets
#Functions for checking irreducibility/finding atoms by referencing our lists of atoms

def is_atom_ref(X):
    for m in range(1,int(max(X)/2+1)):
        for a in atom_gen(m):
            if divides(X,a):
                return False
        file.close()
    return True

def atoms_in_ref(X):
    for S in nonunits(X):
        if is_atom_ref(S):
            yield S

def rec_atoms_ref(n):
    file = open(atom_ref_path+str(n),'w')
    for S in interval_subsets(n):
        if is_atom_ref(S):
            file.write(str(S)+'\n')
    file.close()

In [7]:
#Functions involving partitions

def partitions1(n): #integer partitions of n, code borrowed from [2] above
    a = [0 for i in range(n + 1)]
    k = 1
    y = n - 1
    while k != 0:
        x = a[k - 1] + 1
        k -= 1
        while 2 * x <= y:
            a[k] = x
            y -= x
            k += 1
        l = k + 1
        while x <= y:
            a[k] = x
            a[l] = y
            yield tuple(sorted(a[:k + 2],reverse=True))
            x += 1
            y -= 1
        a[k] = x + y
        y = x + y - 1
        yield tuple(sorted(a[:k + 1],reverse=True))

def partitions(n): #returns list of partitions of n, sorted by max
    return sorted([tuple(sorted(p,reverse=True)) for p in partitions1(n)],reverse=True)

def parts(n,k): #returns list of partitions of n into exactly k parts, sorted by max
    return [P for P in partitions(n) if len(P)==k]

In [8]:
#Functions for finding factorizations, using partition type

def facs_of_type(X,p): #returns all factorizations of X of p-type p
    F = set()
    atom_classes = []
    for m in p:
        atom_classes.append(atoms_in_gen(m,X))
    atom_tuples = it.product(*atom_classes)
    for aa in atom_tuples:
        if msum(*aa) == X:
            F.add(aa)
    return set(tuple(sorted(f,reverse=True)) for f in F)


def feasible_types(X): #returns generator with partitions which could occur as types of S
    for p in partitions1(max(X)):
        if is_subset(subsums(p),X):
            yield p

def facs_by_type(X): #returns a list of factorizations of X, assembled according to feasible p-types
    F = set()
    for p in feasible_types(X):
        F = F.union(facs_of_type(X,p))
    return F

def facs_by_force(X): #returns the set of factorizations of X into atoms, not using partition types of X
    if is_atom(X):
        return {(tuple(sorted(X)),)}
    facs = set()
    for A in atoms_dividing(X):
        for B in nonunits(cofactor(X,A)):
            if setsum(A,B) == X:
                for f in facs_by_force(B):
                    facs.add(tuple(sorted(
                        (tuple(sorted(A)),)+f)))
    return facs

def lengths(X):
    return set(len(f) for f in facs_by_type(X))

In [ ]:
#The cells below have some 'experiments' from various questions I was pondering.  

In [46]:
n = 15
Y = set(range(n+1))
for m in range(1,10):
    print(set(len(f) for f in facs_by_type(Y-{m})))

{2, 3, 4, 5, 6, 7}
{2, 3, 4, 5}
{2, 3, 4, 5}
{2, 3, 4, 5}
{2, 3, 4, 5}
{2, 3, 4, 5, 6}
{2, 3, 4, 5, 6, 7}
{2, 3, 4, 5, 6, 7}
{2, 3, 4, 5, 6}


In [44]:
X = set(range(20))-{6}
Y = set(range(51))-{15}
Z = setsum(X,Y)
len(Z),max(Z)

(70, 69)

In [76]:
def haslen2(X):
    for m in range(1,int(.5*max(X)+2)):
        n = max(X) - m
        f = open(atom_ref_path+str(m),'r')
        g = open(atom_ref_path+str(n),'r')
        at_classes = [[tuple(eval(a)) for a in f if is_subset(eval(a),X)],
                      [tuple(eval(a)) for a in g if is_subset(eval(a),X)]]
        f.close()
        g.close()
        at_tuples = it.product(*at_classes)
        for aa in at_tuples:
            if len(msum(*aa)) == X:
                if msum(*aa) == X:
                    return True
    return False

In [77]:
n = 10
trips = []
t = time()
for p in parts(n,3):
    print(time() - t)
    t = time()
    if 0 not in p:
        f = open(atom_ref_path+str(p[0]))
        g = open(atom_ref_path+str(p[1]))
        h = open(atom_ref_path+str(p[2]))
        at_classes = [[tuple(eval(a)) for a in f],
                      [tuple(eval(a)) for a in g],
                      [tuple(eval(a)) for a in h]]
        f.close()
        g.close()
        h.close()
        at_tuples = it.product(*at_classes)
        for aa in at_tuples:
            if not haslen2(msum(*aa)):
                trips.append(aa)

0.0003902912139892578
2.1457672119140625e-06
0.8071022033691406
9.5367431640625e-07
0.36905598640441895
0.3880767822265625
0.529757022857666
0.11855173110961914
0.5271868705749512
0.3993511199951172
0.2071077823638916
0.2097930908203125
0.420712947845459
0.20154809951782227
0.37605786323547363


In [78]:
goods = list(set(tuple(msum(*aa)) for aa in trips))
goods = [set(g) for g in goods]

In [79]:
len(goods)

40

In [80]:
glens = [lengths(g) for g in goods]

In [81]:
glens

[{2, 3, 4, 5},
 {2, 3, 4},
 {2, 3, 4},
 {2, 3},
 {2, 3},
 {3},
 {3},
 {2, 3},
 {2, 3},
 {2, 3},
 {3},
 {2, 3, 4},
 {2, 3},
 {2, 3},
 {2, 3, 4},
 {2, 3, 4},
 {2, 3},
 {2, 3, 4, 5},
 {3},
 {2, 3},
 {2, 3},
 {2, 3},
 {2, 3, 4},
 {2, 3},
 {2, 3},
 {2, 3},
 {2, 3},
 {2, 3},
 {2, 3},
 {2, 3, 4},
 {2, 3},
 {2, 3},
 {2, 3},
 {2, 3},
 {2, 3, 4},
 {3},
 {2, 3, 4, 5, 6, 7, 8, 9, 10},
 {2, 3, 4},
 {3},
 {2, 3}]

In [86]:
X = goods[1]
X

{0, 1, 2, 3, 4, 7, 8, 9, 10}

In [87]:
F = facs_by_type(X)

In [88]:
len(F)

7

In [ ]:
#The following contracted cells were for the purpose of 
#determining whether subsets of \NN always have a unique longest factorization
#They do not; the suppressed code produces the counterexamples with max = 10

In [9]:
SS = interval_subsets(10)

In [11]:
TT = [X for X in SS if not is_atom(X)]

In [13]:
len(TT)

189

In [14]:
FF = [facs_by_type(X) for X in TT]

In [15]:
len(FF)

189

In [17]:
count = 0
for F in FF:
    if len(F)==1:
        count +=1
print(count)

107


In [18]:
Tnu = []
Fnu = []
for i in range(len(TT)):
    if len(FF[i]) > 1:
        Tnu.append(TT[i])
        Fnu.append(FF[i])
        

In [26]:
Mnu = []
for i in range(len(Tnu)):
    Mnu.append(max([len(f) for f in Fnu[i]]))

In [28]:
VV = []
GG = []
for i in range(len(Tnu)):
    if len([f for f in Fnu[i] if len(f)==Mnu[i]]) > 1:
        VV.append(Tnu[i])
        GG.append(Fnu[i])

In [32]:
LL = [set(len(f) for f in GG[i]) for i in range(len(GG))]

In [64]:
print('i  :  L(X) : #longest factorizations')
for i in range(len(VV)):
    print(i,' : ',LL[i],' : ',len([f for f in GG[i] if len(f)==max([len(f) for f in GG[i]])]))

i  :  L(X) : #longest factorizations
0  :  {2}  :  2
1  :  {2}  :  2
2  :  {2}  :  2
3  :  {2}  :  2
4  :  {2}  :  3
5  :  {2}  :  3
6  :  {2}  :  2
7  :  {2}  :  5
8  :  {2}  :  2
9  :  {2}  :  5
10  :  {2, 3}  :  2
11  :  {2}  :  2
12  :  {2}  :  2
13  :  {2}  :  3
14  :  {2}  :  2
15  :  {2, 3}  :  2
16  :  {2}  :  2
17  :  {2}  :  3
18  :  {2}  :  3
19  :  {2}  :  2
20  :  {2}  :  3
21  :  {2}  :  2
22  :  {2}  :  5
23  :  {2}  :  5
24  :  {2}  :  3
25  :  {2}  :  3
26  :  {2}  :  5
27  :  {2}  :  2
28  :  {2}  :  5
29  :  {2}  :  2
30  :  {2}  :  2
31  :  {2}  :  2
32  :  {2}  :  8
33  :  {2, 3}  :  4
34  :  {2}  :  9
35  :  {2}  :  8
36  :  {2, 3}  :  4
37  :  {2}  :  2
38  :  {2}  :  4
39  :  {2}  :  2
40  :  {2}  :  2
41  :  {2}  :  3
42  :  {2}  :  2
43  :  {2}  :  2
44  :  {2, 3}  :  4
45  :  {2}  :  6
46  :  {2}  :  2
47  :  {2}  :  4
48  :  {2}  :  9
49  :  {2}  :  2
50  :  {2}  :  2
51  :  {2}  :  8
52  :  {2, 3}  :  4
53  :  {2}  :  8
54  :  {2, 3, 4}  :  4
55  :  {2, 3} 

In [67]:
for i in range(len(VV)):
    print('i =',i,': ',VV[i])
    print()
    G = GG[i]
    m = max([len(g) for g in G])
    for f in G:
        if len(f) == m:
            print(f)
    print()

i = 0 :  {0, 1, 2, 3, 9, 10}

((0, 9, 2), (0, 1))
((0, 1, 2, 9), (0, 1))

i = 1 :  {0, 1, 3, 4, 7, 10}

((0, 3), (0, 1, 4, 7))
((0, 3), (0, 1, 7))

i = 2 :  {0, 1, 7, 8, 9, 10}

((0, 9, 7), (0, 1))
((8, 0, 9, 7), (0, 1))

i = 3 :  {0, 3, 6, 7, 9, 10}

((0, 3, 6, 7), (0, 3))
((0, 6, 7), (0, 3))

i = 4 :  {0, 1, 2, 3, 4, 9, 10}

((0, 1, 2, 3, 9), (0, 1))
((0, 9, 2, 3), (0, 1))
((0, 1, 3, 9), (0, 1))

i = 5 :  {0, 1, 6, 7, 8, 9, 10}

((0, 6, 7, 8, 9), (0, 1))
((0, 9, 6, 7), (0, 1))
((8, 0, 6, 9), (0, 1))

i = 6 :  {0, 1, 2, 3, 4, 5, 7, 10}

((0, 3), (0, 1, 2, 4, 7))
((0, 3), (0, 1, 2, 7))

i = 7 :  {0, 1, 2, 3, 4, 5, 9, 10}

((0, 1, 2, 4, 9), (0, 1))
((0, 1, 2, 3, 4, 9), (0, 1))
((0, 9, 2, 4), (0, 1))
((0, 1, 3, 4, 9), (0, 1))
((0, 2, 3, 4, 9), (0, 1))

i = 8 :  {0, 1, 2, 3, 4, 6, 7, 10}

((0, 2, 3, 6), (0, 1, 4))
((0, 2, 6), (0, 1, 4))

i = 9 :  {0, 1, 2, 3, 4, 6, 8, 10}

((0, 2), (0, 1, 4, 6, 8))
((0, 2), (0, 1, 2, 4, 8))
((0, 2), (0, 1, 4, 8))
((0, 2), (0, 1, 2, 6, 8))
((0, 2), (0, 1, 

In [44]:
i = 10
G = GG[i]
m = max([len(g) for g in G])
for f in G:
    if len(f) == m:
        print(f)


((0, 8, 2), (0, 1), (0, 1))
((0, 1, 2, 8), (0, 1), (0, 1))


In [46]:
X = {0, 1, 2, 3, 5, 7, 8, 10}

[2, 2, 2]


In [53]:
msum((0, 2, 7), (0, 1, 3)), msum((0, 2), (0, 1, 5, 8)), msum((0, 2), (0, 1, 3, 5, 8))

({0, 1, 2, 3, 5, 7, 8, 10},
 {0, 1, 2, 3, 5, 7, 8, 10},
 {0, 1, 2, 3, 5, 7, 8, 10})

In [54]:
X = {0,1,2,3,5,7,8,10}

In [55]:
F = facs_by_type(X)

In [56]:
for f in F:
    print(f)

((0, 2, 7), (0, 1, 3))
((0, 2), (0, 1, 5, 8))
((0, 2), (0, 1, 3, 5, 8))


In [9]:
#After this cell we begin to look into the question of how much duplication there is among sums atoms
#In particular, we consider sums of two atoms

In [10]:
def nonatoms_of_type(n,P): #produces nonatoms with a factorization of type P and max n
    sums = set()
    atom_classes = []
    for m in P:
        atom_classes.append(atom_gen(m))
    for aa in it.product(*atom_classes):
        sums.add(tuple(sorted(msum(*aa))))
    return sums

def exp_nonatoms_of_type(n,P):
    N = 1
    for m in P:
        N = N*len([a for a in atom_gen(m)])
    return N

def nonatoms_of_len(n,l):
    sums = set()
    for P in parts(n,l):
        sums = sums.union(nonatoms_of_type(n,P))
    return sums

def exp_nonatoms_of_len(n,l):
    return sum([exp_nonatoms_of_type(n,P) for P in parts(n,l)])

In [13]:
n=10
p=5
P = (p,n-p)
N = nonatoms_of_type(n,P)
E = exp_nonatoms_of_type(n,P)
len(N),E

(32, 81)

In [19]:
NA = {}
for n in N:
    NA[n] = 0
for a in atom_gen(5):
    for b in atom_gen(5):
        NA[tuple(sorted(msum(a,b)))] += 1
        

In [22]:
for n in NA:
    if NA[n] >= 4:
        print(n,':',NA[n])

(0, 1, 2, 3, 5, 6, 7, 10) : 4
(0, 3, 4, 5, 7, 8, 9, 10) : 4
(0, 1, 3, 4, 5, 6, 7, 8, 9, 10) : 4
(0, 1, 2, 3, 4, 5, 6, 7, 9, 10) : 4
(0, 2, 3, 4, 5, 6, 7, 8, 9, 10) : 4
(0, 1, 3, 4, 5, 6, 8, 10) : 4
(0, 1, 2, 3, 4, 5, 6, 7, 8, 10) : 4
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10) : 4
(0, 2, 4, 5, 6, 7, 9, 10) : 4


In [25]:
NA4 = [n for n in NA if NA[n] == 4]

In [44]:
X = set(NA4[4])
F = facs_by_type(X)
print(X)
print(len(F))
print()
for f in F:
    print(f)

{0, 2, 3, 4, 5, 6, 7, 8, 9, 10}
49

((0, 3, 4), (0, 3), (0, 2, 3))
((0, 2, 5, 6, 7), (0, 2, 3))
((0, 3, 4, 5), (0, 2, 3), (0, 2))
((0, 3, 4), (0, 2, 3, 4), (0, 2))
((0, 4, 5), (0, 2, 3), (0, 2))
((0, 3, 4, 5, 6), (0, 2, 3, 4))
((0, 3, 4, 5, 6), (0, 2), (0, 2))
((0, 3, 4), (0, 2, 4, 5, 6))
((0, 2, 3, 6), (0, 2, 3, 4))
((0, 4), (0, 2, 3), (0, 2, 3))
((0, 3, 5), (0, 2, 3), (0, 2))
((0, 2, 3, 4, 6), (0, 2, 3, 4))
((0, 2, 4, 5), (0, 2, 3), (0, 2))
((0, 2, 3, 6, 7), (0, 2, 3))
((0, 2, 4, 6, 7), (0, 2, 3))
((0, 3, 4, 5), (0, 2, 5))
((0, 2, 3, 4), (0, 2, 3), (0, 2, 3))
((0, 2, 3), (0, 2, 3), (0, 2), (0, 2))
((0, 3, 4, 5, 7), (0, 2, 3))
((0, 4, 5), (0, 3), (0, 2))
((0, 4, 5, 6, 7), (0, 2, 3))
((0, 5, 6), (0, 2, 3, 4))
((0, 4), (0, 2, 3, 4), (0, 2))
((0, 3), (0, 2, 4, 5), (0, 2))
((0, 2, 3, 5, 6, 7), (0, 2, 3))
((0, 3, 4), (0, 3, 4), (0, 2))
((0, 2, 3, 4, 7, 8), (0, 2))
((0, 3, 4), (0, 2, 3), (0, 2, 3))
((0, 4, 5, 7), (0, 2, 3))
((0, 2, 4, 5, 6), (0, 2, 3, 4))
((0, 2, 3, 6, 7, 8), (0, 2))
((0, 4

In [52]:
#Number of Interval Factorizations
zz = []
for n in range(2,15):
    zz.append(len(facs_by_type(set(range(n+1))-{-1})))

In [53]:
zz

[1, 2, 4, 8, 16, 37, 81, 183, 424, 986, 2288, 5316, 12374]

In [149]:
n=12
l=2
N = nonatoms_of_len(n,l)
E = exp_nonatoms_of_len(n,l)
len(N),E

(609, 2957)

In [210]:
n=16
l = 2
NN = 0
EE = 0
print('P       N     E     N/E')
for P in parts(n,l):
    N = len(nonatoms_of_type(n,P))
    E = exp_nonatoms_of_type(n,P)
    NN += N
    EE += E
    print(P,N,E,round(N/E,2))
print()
print(len(nonatoms_of_len(n,l)),NN,EE,NN/EE, len(parts(n,l))*2**(n-l))

P       N     E     N/E
(15, 1) 1834 12368 0.15
(14, 2) 1280 5956 0.21
(13, 3) 1532 8706 0.18
(12, 4) 1227 7120 0.17
(11, 5) 1313 6138 0.21
(10, 6) 1258 6137 0.2
(9, 7) 1295 6552 0.2
(8, 8) 661 5929 0.11

7371 10400 58906 0.17655247343224797 131072


In [197]:
N = nonatoms_of_type(12,(6,6))

In [200]:
lens = {}
for n in N:
    if len(n) not in lens:
        lens[len(n)]=1
    else:
        lens[len(n)]+=1
lens

{10: 16, 7: 10, 6: 4, 12: 8, 11: 19, 9: 18, 5: 4, 8: 4, 13: 1, 3: 1}

In [74]:
n = 12
sums = set()
for q in range(1,7):
    for a in atom_ref(n-q):
        for b in atom_ref(q):
            sums.add(tuple(sorted(msum(a,b))))

In [75]:
len(sums)

2035

In [86]:
n = 12
exp_sums = 0
for q in range(1,int(n/2+1)):
    exp_sums += len([a for a in atom_ref(n-q)])*len([b for b in atom_ref(q)])
    

In [9]:
#counting factorizations of intervals

In [13]:
n=10
I = set(range(n+1))
Z = facs_by_type(I)
print(len(Z))

424


In [51]:
def intfacs_max(n,m): #returns factorizations of [0,n] which include an atom with max m
    types = [P for P in partitions(n) if m in P]
    facs = set()
    for P in types:
        facs = facs.union(facs_of_type(set(range(n+1)),P))
    return facs
        

In [84]:
for N in range(5,20):
    n=N
    z = len(facs_by_type(set(range(n+1))))
    z1 = len(intfacs_max(n,1))
    zh = len(intfacs_max(n,math.floor(n/2)))
    
    print('n =',n)
    print('z:', z)
    print('z1:', z1)
    print('zh:', zh)
    print('z1/z:',float(z1)/float(z))
    print()

n = 5
z: 8
z1: 8
zh: 2
z1/z: 1.0

n = 6
z: 16
z1: 15
zh: 7
z1/z: 0.9375

n = 7
z: 37
z1: 35
zh: 13
z1/z: 0.9459459459459459

n = 8
z: 81
z1: 75
zh: 25
z1/z: 0.9259259259259259

n = 9
z: 183
z1: 168
zh: 48
z1/z: 0.9180327868852459

n = 10
z: 424
z1: 375
zh: 90
z1/z: 0.8844339622641509

n = 11
z: 986
z1: 855
zh: 185
z1/z: 0.8671399594320487

n = 12
z: 2288
z1: 1946
zh: 445
z1/z: 0.8505244755244755

n = 13
z: 5316
z1: 4427
zh: 986
z1/z: 0.8327689992475545

n = 14
z: 12374
z1: 10107
zh: 2218
z1/z: 0.8167932762243414

n = 15
z: 28792
z1: 23079
zh: 4814
z1/z: 0.8015768268963601

n = 16
z: 66784
z1: 52594
zh: 10193
z1/z: 0.7875239578342118

n = 17
z: 154688
z1: 119761
zh: 22877
z1/z: 0.774210022755482

n = 18
z: 357359
z1: 272144
zh: 52683
z1/z: 0.7615423145912094

n = 19
z: 824002
z1: 617696
zh: 118492
z1/z: 0.7496292484726979



In [85]:
for N in range(20,26):
    n=N
    z = len(facs_by_type(set(range(n+1))))
    z1 = len(intfacs_max(n,1))
    zh = len(intfacs_max(n,math.floor(n/2)))
    
    print('n =',n)
    print('z:', z)
    print('z1:', z1)
    print('zh:', zh)
    print('z1/z:',float(z1)/float(z))
    print()

KeyboardInterrupt: 

In [87]:
for n in range(2,18):
    print('n =',n,':','|Z| =',len(facs_by_type(set(range(n+1)))))

n = 2 : |Z| = 1
n = 3 : |Z| = 2
n = 4 : |Z| = 4
n = 5 : |Z| = 8
n = 6 : |Z| = 16
n = 7 : |Z| = 37
n = 8 : |Z| = 81
n = 9 : |Z| = 183
n = 10 : |Z| = 424
n = 11 : |Z| = 986
n = 12 : |Z| = 2288
n = 13 : |Z| = 5316
n = 14 : |Z| = 12374
n = 15 : |Z| = 28792
n = 16 : |Z| = 66784
n = 17 : |Z| = 154688
